In [2]:
import math, copy, random, operator
import numpy as np
from numpy.linalg import norm
from gaft.components import BinaryIndividual
from gaft.components import DecimalIndividual
from gaft.components import Population
from gaft.operators import TournamentSelection
from gaft.operators import UniformCrossover
from gaft.operators import FlipBitMutation
from gaft.analysis import ConsoleOutput
from gaft.analysis.fitness_store import FitnessStore
from gaft import GAEngine
from gaft.plugin_interfaces.analysis import OnTheFlyAnalysis
import matplotlib.pyplot as plt

#############################################################################
### Setting simulation parameters
MaliciousNodeRatio = [0.01,0.03,0.05,0.07,0.09,0.11,0.13,0.15,0.17,0.19,0.21,0.23,0.25,0.27,0.29,0.31,0.33,0.35,0.37,0.39,0.41,0.43,0.45,0.47,0.49]
Iteration = 10 # The number of Iteration for simulation
RoundNumber = 10 # The number of consensus round
ShardNum = 10 # The number of shard
ShardSize = 20 # The number of people in a shard
NodeNum = ShardNum * ShardSize # Total number of simulation
AttackType = 3 # NMA = 1 , CRA = 2 , CBA = 3
GenerationNumber = 300 # The number of generation in Genetic Algorithm
#############################################################################

### function for cosine similarity
def cos_sim(A,B):
    return np.dot(A,B) / (norm(A)*norm(B)) 

### function for GA
def GA(GenerationNumber,NodeTrust):

    InitialParents = []
    for i in range(NodeNum):
        InitialParents.append((0,ShardNum-1))
    indv = DecimalIndividual(ranges=InitialParents, eps=1)
    population = Population(indv_template=indv, size=100).init() # Setting the number of population
    selection = TournamentSelection() # Setting the way of selection
    crossover = UniformCrossover(pc=0.8, pe=0.5) # Setting the probability of crossover
    mutation = FlipBitMutation(0.1,NodeTrust, ShardNum) # Setting the probability of mutation
    engine = GAEngine(population=population, selection=selection,
                      crossover=crossover, mutation=mutation,
                      )
    
    SSEList = []
    @engine.fitness_register
    @engine.minimize
    def fitness(indv): # Function for fitness function
        XX = []
        XX = indv.solution
        tmp = 0
        YY = []
        ZZ = []
        for i in range(ShardNum):
            YY.append(0)
            ZZ.append(0)
        for i in range(NodeNum):
            YY[XX[i]] += NodeTrust[i]
            ZZ[XX[i]] += 1
        SSE = 0
        SSE1 = 0
        SSE2 = 0
        for i in range(ShardNum):
            SSE1 += (YY[i]-TotalTrustAverage)**2
            SSE2 += (ZZ[i]-(NodeNum/ShardNum))**2
        SSE1 = float((SSE1/ShardNum)**(1/2))
        SSE2 = float((SSE2/ShardNum)**(1/2))
        SSE = SSE1*SSE2
        SSEList.append(SSE)
        return SSE
    
    global gList
    global fitnessList

    gList = []
    fitnessList = []
    
    
    @engine.analysis_register
    class ConsoleOutput(OnTheFlyAnalysis):
        master_only = True
        interval = 1
        def register_step(self, g, population, engine):
            best_indv = population.best_indv(engine.fitness)
            msg = 'Generation: {}, best fitness: {:.3f}'.format(g, -engine.fmax)
            gList.append(g)
            fitnessList.append(-engine.fmax)
            #engine.logger.info(msg)
    engine.run(ng=GenerationNumber)
    best_indv = engine.population.best_indv(engine.fitness)
    best_indv.solution
    TotgList.append(gList)
    TotfitnessList.append(fitnessList)
    
    return best_indv.solution

RoundfitnessList = []

### Main Function
for MalRatio in MaliciousNodeRatio:

    MaliciousNum = round(NodeNum*MalRatio) # The number of malicious people
    FalsificationNum = 0 # The number of falsification malicious people
    NodeType = {}
    Falsification = {}
    TotgList = []
    TotfitnessList = []
    if ((AttackType == 2) or (AttackType == 3)):
        FalsificationNum = MaliciousNum
        
    ### If NodeType is 0, it is malicious node
    for i in range(MaliciousNum):
        NodeType[i] = 0
    for i in range(MaliciousNum,NodeNum):
        NodeType[i] = 1
        
    ### If Falsification is 1, It falisify LCR
    for i in range(FalsificationNum):
        Falsification[i] = 1
    for i in range(FalsificationNum,NodeNum):
        Falsification[i] = 0       
    
    # Variable for simulation data
    A = []
    B = []
    C = []
    D = []
    E = []
    F = []
    G = []
    TP = []
    TN = []
    FP = []
    FN = []
    TPHonest = []
    TNHonest = []
    AvgfitnessList = []
    
    for iterations in range(Iteration): # Set iterations for simulation
        global NodeTrust
        ### Set initial trust value of node to be 1
        NodeTrust = {}
        for i in range(NodeNum):
            NodeTrust[i] = 1

        TPCountList = [] # Wrong Block & Block Making X
        FNCountList = [] # Wrong Block & Block Making O
        FPCountList = [] # Right Block & Block Making X
        TNCountList = [] # Right Block & Block Making O
        TPHonestCountList = [] # Wrong Block & Block Making X by Honest Nodes
        TNHonestCountList = [] # Right Block & Block Making O by Honest Nodes
        BadShardList = [] # The number of Shard consisting of 50% malicious node

        ### Start consensus round
        for Round in range(RoundNumber): # Set the number of consensus round

            NodeShard = {} # Represents shard number of nodes
            ShardCount = {}
            TPCount = 0
            FNCount = 0
            FPCount = 0
            TNCount = 0
            TPHonestCount = 0
            TNHonestCount = 0
            BadShard = 0
            for i in range(ShardNum):
                ShardCount[i] = 0
            TotalTrustAverage = 0
            tmp = 0
            
            if (Round == 0): # It is initial round (random distribution)
                for i in range(NodeNum):
                    tmp = random.randint(0,ShardNum-1)
                    while (ShardCount[tmp] >= ShardSize):
                        tmp = random.randint(0, ShardNum-1)  
                    NodeShard[i] = tmp
                    ShardCount[tmp] += 1

            else :  # After first round, shard of node is set by using GA
                for i in range(NodeNum):
                    TotalTrustAverage += NodeTrust[i]
                TotalTrustAverage = TotalTrustAverage/ShardNum
           
                GAResult = GA(GenerationNumber,NodeTrust)
                for i in range(NodeNum):
                    NodeShard[i] = GAResult[i]


            # Saves index of node according ot shard number
            ShardNode = {}
            for K in range(ShardNum):
                ShardNode[K] = []
            for i in range(NodeNum):
                ShardNode[NodeShard[i]].append(i)

            # Selecting miner randomly in each shard
            Miner = {}
            BlockHF = {}
            for i in range(ShardNum):
                tmp = random.randint(0,len(ShardNode[i])-1)
                Miner[i] = ShardNode[i][tmp]
                if (NodeType[Miner[i]] == 1):         
                    BlockHF[i] = 1
                else:
                    BlockHF[i] = 0


            # Saves nodes' pros or cons in each shard
            ShardConsensus = {}
            for K in range(ShardNum):
                ShardConsensus[K] = []
            # NMA, CRA votes 'pros' or 'cons' reversly
            if ((AttackType == 1) or (AttackType == 2)):
                for i in range(NodeNum):
                    if ((NodeType[i] == 1) & (BlockHF[NodeShard[i]] == 0)):
                        ShardConsensus[NodeShard[i]].append(0)
                    elif ((NodeType[i] == 1) & (BlockHF[NodeShard[i]] == 1)):
                        ShardConsensus[NodeShard[i]].append(1)
                    elif ((NodeType[i] == 0) & (BlockHF[NodeShard[i]] == 0)):
                        ShardConsensus[NodeShard[i]].append(1)
                    elif ((NodeType[i] == 0) & (BlockHF[NodeShard[i]] == 1)):
                        ShardConsensus[NodeShard[i]].append(0)  
            # CBA votes randomly
            elif (AttackType == 3) :
                for i in range(NodeNum):
                    if ((NodeType[i] == 1) & (BlockHF[NodeShard[i]] == 0)):
                        ShardConsensus[NodeShard[i]].append(0)
                    elif ((NodeType[i] == 1) & (BlockHF[NodeShard[i]] == 1)):
                        ShardConsensus[NodeShard[i]].append(1)
                    elif ((NodeType[i] == 0) & (BlockHF[NodeShard[i]] == 0)):
                        ShardConsensus[NodeShard[i]].append(1)
                    elif ((NodeType[i] == 0) & (BlockHF[NodeShard[i]] == 1)):
                        if (random.randint(0,1) == 1):
                            ShardConsensus[NodeShard[i]].append(0)
                        else:
                            ShardConsensus[NodeShard[i]].append(1)
                              


            LCR = {} # Local Consensus Result
            U = {} # Average of LCR
            for K in range(ShardNum):
                LCR[K] = np.zeros((len(ShardConsensus[K]),len(ShardConsensus[K])))
                U[K] = np.zeros((1,len(ShardConsensus[K])))

            for K in range(ShardNum):
                pros = 0
                cons = 0
                pros_false = 0
                cons_false = 0
                pros_cnt = 0
                cons_cnt = 0
                
                I = 0
                for i in range(len(ShardConsensus[K])):
                    if (ShardConsensus[K][i] == 1):
                        pros += 1
                    else:
                        cons += 1

                mal = 0
                hon = 0
                for i in ShardNode[K]:
                    if (NodeType[i] == 1):
                        hon += 1
                    else:
                        mal += 1
                # if (malicious >= honest) then malicious nodes occupy their shard
                if (hon <= mal):
                    BadShard += 1            

                # Judge TN,TP,FN,FP
                if ((pros > ((1/2)*(pros+cons))) & (NodeType[Miner[K]] == 1)):
                    if (mal < hon):
                        TNHonestCount += 1                    
                    TNCount += 1
                elif ((pros <= ((1/2)*(pros+cons))) & (NodeType[Miner[K]] == 1)):
                    FPCount += 1
                elif ((pros > ((1/2)*(pros+cons))) & (NodeType[Miner[K]] == 0)):
                    FNCount += 1                
                elif ((pros <= ((1/2)*(pros+cons))) & (NodeType[Miner[K]] == 0)):
                    if (mal < hon):
                        TPHonestCount += 1
                    TPCount += 1                          

                # Each nodes evaluates LCR (nodes' trust value in their shard)
                for i in range(NodeNum):
                    if (NodeShard[i] == K):
                        # Honest nodes evaluate honestly
                        if (NodeType[i] == 1): 
                            for j in range(len(ShardConsensus[K])):
                                if (ShardConsensus[K][j] == 1):
                                    LCR[K][I,j] = pros/(pros+cons)
                                else:
                                    LCR[K][I,j] = cons/(pros+cons)    
                        # Malicious and no falsification nodes evaluate honestly           
                        elif ((NodeType[i] == 0) & (Falsification[i] == 0)):
                            for j in range(len(ShardConsensus[K])):
                                if (ShardConsensus[K][j] == 1):
                                    LCR[K][I,j] = pros/(pros+cons)
                                else:
                                    LCR[K][I,j] = cons/(pros+cons)
                        # Malicious and falsification nodes evaluate falsy                     
                        elif ((NodeType[i] == 0) & (Falsification[i] == 1)):
                            pros_false = random.randint(0, len(ShardConsensus[K]))
                            cons_false = len(ShardConsensus[K]) - pros_false
                            pros_cnt = 0
                            cons_cnt = 0                            
                            if ((pros_false == 0) or (cons_false == 0)):
                                for j in range(len(ShardConsensus[K])):
                                    LCR[K][I,j] = 1
                            else:
                                for j in range(len(ShardConsensus[K])):
                                    if (pros_cnt == len(ShardConsensus[K])):
                                        LCR[K][I,j] = cons_false/(pros_false+cons_false)
                                    elif (cons_cnt == len(ShardConsensus[K])):
                                        LCR[K][I,j] = pros_false/(pros_false+cons_false)
                                    else:
                                        if (random.randint(0,1) == 0):
                                            LCR[K][I,j] = pros_false/(pros_false+cons_false)
                                            pros_cnt += 1
                                        else :
                                            LCR[K][I,j] = cons_false/(pros_false+cons_false)
                                            cons_cnt += 1
                        I += 1


            BadShardList.append(BadShard/ShardNum)
            TPCountList.append(TPCount)
            FNCountList.append(FNCount)
            FPCountList.append(FPCount)
            TNCountList.append(TNCount)
            TPHonestCountList.append(TPHonestCount)
            TNHonestCountList.append(TNHonestCount)

            NodeU = {} # Nodes' Average LCR
            NodeW = {} # Nodes' cosine similarity

            # Calculate average LCR
            for K in range(ShardNum):
                J = 0
                for i in range(NodeNum):
                    u = 0
                    if (NodeShard[i] == K):
                        for I in range(len(ShardConsensus[K])):
                            u += LCR[K][I,J]
                        u = u/len(ShardConsensus[K])

                        NodeU[i] = u
                        U[K][0,J] = u
                        J += 1
            # Calculate cosine similarity and trust value
            for K in range(ShardNum):
                I = 0
                for i in range(NodeNum):
                    W = 0
                    if (NodeShard[i] == K):
                        for J in range(len(ShardConsensus[K])):
                            W += cos_sim(LCR[K][I],LCR[K][J])
                        W = W/len(ShardConsensus[K])
                        NodeW[i] = W
                        Trust = W*NodeU[i]
                        NodeTrust[i] = (NodeTrust[i]*Round+Trust)/(Round+1)
                        I += 1                

        SuccessRatio = (sum(TPHonestCountList) + sum(TNHonestCountList))/(RoundNumber*ShardNum)

            
        A.append(SuccessRatio)
        B.append(sum(BadShardList)/len(BadShardList))
        C.append(TPCountList)
        D.append(TNCountList)
        F.append(TPHonestCountList)
        G.append(TNHonestCountList)
        TP.append(sum(TPCountList))
        FN.append(sum(FNCountList))
        FP.append(sum(FPCountList))
        TN.append(sum(TNCountList))
        TPHonest.append(sum(TPHonestCountList))
        TNHonest.append(sum(TNHonestCountList))
    # Accuracy according to round number
    for i in range(RoundNumber):
        tmp = 0
        for j in range(Iteration):
            tmp += (F[j][i] + D[j][i])
        tmp = tmp/(Iteration*ShardNum)
        E.append(tmp)
    # Fitness function values
    for i in range(GenerationNumber):
        tmp = 0
        for j in range(len(TotfitnessList)):
            tmp += (TotfitnessList[j][i])
        tmp = tmp/len(TotfitnessList)
        AvgfitnessList.append(tmp)    

  
    
    print('MaliciousNodeRatio is '+str(MalRatio))
    print('# of Malicious Shard is '+str(sum(B)/len(B)))
    print('# of TP is '+str(sum(TPHonest)/len(TPHonest)))
    print('# of FN is '+str(sum(FN)/len(FN)))
    print('# of FP is '+str(sum(FP)/len(FP)))
    print('# of TN is '+str(sum(TNHonest)/len(TNHonest)))
    print(E)
    print()


MaliciousNodeRatio is 0.41
# of Malicious Shard is 0.25
# of TP is 21.5
# of FN is 9.5
# of FP is 0.0
# of TN is 53.5
[0.95, 0.95, 0.85, 0.85, 0.8, 0.9, 0.7, 0.95, 0.9, 0.8]

MaliciousNodeRatio is 0.43
# of Malicious Shard is 0.31499999999999995
# of TP is 23.0
# of FN is 14.0
# of FP is 0.0
# of TN is 45.5
[0.75, 0.8, 0.75, 0.75, 0.8, 0.85, 0.8, 0.8, 0.85, 0.75]



In [ ]:
NodeTrust